In [ ]:
!pip install transformers datasets evaluate accelerate simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 3

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import nltk
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
nltk.download("movie_reviews")

from collections import defaultdict, Counter
import math
import random

random.seed(0) # Don't change
torch.manual_seed(0)  # Don't change
np.random.seed(0) # Don't change

train_X, train_Y = [], []
test_X, test_Y = [], []

for polarity in movie_reviews.categories():
    label = 0 if polarity == 'neg' else 1
    for fid in movie_reviews.fileids(polarity):
        if random.randrange(5) == 0:
            test_X.append(movie_reviews.raw(fid))
            test_Y.append(label)
        else:
            train_X.append(movie_reviews.raw(fid))
            train_Y.append(label)
print(train_X[0], train_Y[0])

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn't snag this one correctly . 
they seem to have taken this pretty neat concept , but executed it terribly . 
so what are the problems with the movie ? 
well , its main problem is that it's simply too jumbled . 
it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member , have no id

In [ ]:
from collections import Counter
Counter(train_Y)

Counter({0: 775, 1: 803})

# Assignment III
Doing Assignment III by modifying the following code cell.
Your goal is to train a Transformer based model to achieve an Accuracy of 0.85. The higher, the better.

Your solution should be based on Transformer models.
You are free to adjust [the settings](https://simpletransformers.ai/docs/usage/) of the simpletransformers class and [the pre-trained models](https://huggingface.co/models?sort=trending). You can also build the model with [the transformers package](https://huggingface.co/docs/transformers/en/index) for more control in details, but your model has to follow the output format of [the predict() method](https://simpletransformers.ai/docs/classification-models/) of `simpletransformers` for automatic evaluation.

Please note that your code is expected to be executed in the free version of Colab. So please make sure your model and your training procedure is lightweight enough to be successfully executed by our TAs.

#### Optuna 自動搜尋 hyper-parameter

In [ ]:
!pip install optuna simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')
import spacy
from nltk import ngrams


### Stop Words
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS
# sign_list = [";", ",", ".", "~", ":", "-", "(", ")", "%", "#", "$", "!", "/", "?", "=", "+", "&", "--", "'", '"', '`']
# for sign in sign_list:
#   stop_words.add(sign)
### Lemmatize
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(tokens, ngram_range):
  tokens = [token for token in tokens if token not in stop_words]
  tokens = [token.lower() for token in tokens]
  tokens = [lemmatizer.lemmatize(token) for token in tokens]
  tokens = [stemmer.stem(token) for token in tokens]
  ngram_tokens = []
  for n in range(ngram_range[0], ngram_range[1]+1):
      ngrams_n = ngrams(tokens, n)
      ngram_tokens.extend(['_'.join(grams) for grams in ngrams_n])
  return ngram_tokens


import optuna
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 假設已經定義好 preprocess 函數
# 假設已經有 train_X 和 train_Y 數據

# 分割數據集為訓練集和驗證集
train_X = preprocess(train_X, ngram_range=(1,1))
print(len(train_X))
print(len(train_Y))
train_df, eval_df = train_test_split(pd.DataFrame({'text': train_X, 'labels': train_Y}), test_size=0.1)

def objective(trial):
    # 定義需要優化的超參數
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
    num_train_epochs = trial.suggest_int('num_train_epochs', 3, 10)
    train_batch_size = trial.suggest_categorical('train_batch_size', [16, 32, 64])
    warmup_steps = trial.suggest_int('warmup_steps', 0, 100)

    # 設置模型參數
    model_args = ClassificationArgs(
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        train_batch_size=train_batch_size,
        eval_batch_size=16,
        learning_rate=learning_rate,
        warmup_steps=warmup_steps,
        save_eval_checkpoints=False,
        save_model_every_epoch=False,
        fp16=True,
        reprocess_input_data=True
    )

    # 初始化模型
    model = ClassificationModel(
        "bert",
        "bert-base-uncased",
        args=model_args,
        use_cuda=True
    )

    # 訓練模型
    model.train_model(train_df)

    # 驗證模型
    result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=accuracy_score)

    # 返回驗證集的準確率
    return result['acc']

# 設置日誌級別
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# 創建Optuna學習器
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# 輸出最佳參數
print("Best trial:")
trial = study.best_trial
print(f"Accuracy: {trial.value}")
print("Params:")
for key, value in trial.params.items():
  print(f"{key}: {value}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[I 2024-05-21 05:39:14,668] A new study created in memory with name: no-name-24fb4a70-e54b-4eef-ab95-824f453af59f


1578
1578


<ipython-input-30-cc528cb52fa5>:51: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incom

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

[I 2024-05-21 06:14:03,974] Trial 19 finished with value: 0.7151898734177216 and parameters: {'learning_rate': 2.0498377920053568e-05, 'num_train_epochs': 8, 'train_batch_size': 64, 'warmup_steps': 86}. Best is trial 8 with value: 0.7658227848101266.


Best trial:
Accuracy: 0.7658227848101266
Params:
learning_rate: 4.6540926511288235e-05
num_train_epochs: 9
train_batch_size: 16
warmup_steps: 69


#### 作業程式碼 - Transformer

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')
import spacy
from nltk import ngrams


### Stop Words
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS
### Lemmatize
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(tokens, ngram_range):
  tokens = [token for token in tokens if token not in stop_words]
  tokens = [token.lower() for token in tokens]
  tokens = [lemmatizer.lemmatize(token) for token in tokens]
  tokens = [stemmer.stem(token) for token in tokens]
  ngram_tokens = []
  for n in range(ngram_range[0], ngram_range[1]+1):
      ngrams_n = ngrams(tokens, n)
      ngram_tokens.extend(['_'.join(grams) for grams in ngrams_n])
  return ngram_tokens


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


def build_model(training_instances, training_labels):
  training_instances = preprocess(training_instances, ngram_range=(1,3))
  train_data = zip(training_instances, training_labels)
  train_df = pd.DataFrame(train_data)
  # use_cuda=True if GPU is available
  model = ClassificationModel("bert", "bert-base-uncased", use_cuda=True, args={
      'overwrite_output_dir': True,
      'num_train_epochs': 2,
      'train_batch_size': 64,
      'eval_batch_size': 16,
      'fp16': True,
      'save_eval_checkpoints': False,
      'save_model_every_epoch': False,
      'sliding_window': True
      })

  model.train_model(train_df)
  return model

model = build_model(train_X, train_Y)

[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

  0%|          | 0/1578 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Token indices sequence length is longer than the specified maximum sequence length for this model (914 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (598 > 512). Running this sequence through the model will result in indexing errors


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/220 [00:00<?, ?it/s]

Running Epoch 2 of 2:   0%|          | 0/220 [00:00<?, ?it/s]

In [ ]:
def predict(model, document):
    return model.predict([document])[0][0]

print(predict(model, "this is an uninteresting movie"))
print(predict(model, "a good movie of this year"))


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

1


## Do Evaluation

In [ ]:
correct, total = 0, 0

pred_Y = model.predict(test_X)[0]

for prediction, y in zip(pred_Y, test_Y):
    if prediction == y:
        correct += 1
    total += 1

print("%d / %d = %g" % (correct, total, correct / total))

  0%|          | 0/422 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (597 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/226 [00:00<?, ?it/s]

394 / 422 = 0.933649
